In [1]:
import pandas as pd
from glob import glob
from pypcleaner import convert_excel_time

In [2]:
df = pd.read_hdf('big_multi_index.h5')
scans = pd.read_pickle('scans.p')
imaging_id_codes = pd.read_pickle('imaging_id_codes.p')
id_codes = imaging_id_codes.loc[scans.index,'prefix']
scan_id_df = df.loc[scans.index]
scan_id_df.drop('day_0', axis='columns', level=0, inplace = True)

drop_these = ['bp_three_hr','bp_three_hr_systolic','bp_three_hr_diastolic','bp_three_hr_datetime',
              'bp_seven_hr','bp_seven_hr_systolic','bp_seven_hr_diastolic','bp_seven_hr_datetime',
              'bp_other_hr','bp_other_hr_systolic','bp_other_hr_diastolic','bp_other_hr_datetime',
              'hrt_seven_hr','hrt_seven_hr_rate','hrt_seven_hr_datetime','hrt_other_hr',
              'hrt_other_hr_rate','hrt_other_hr_datetime','surgical_intervention',
              'surgical_intervention_detail','invasive_intervention','invasive_intervention_detail',
              'imaging_intervention','imaging_intervention_detail',
              'other_intervention','other_intervention_detail','medication_given','medication_comment']

scan_id_df.drop([('day_baseline','recent_medical_history',c) for c in drop_these], axis = 'columns',inplace = True)

scan_id_df[('day_baseline','recent_medical_history','tpa_datetime')] = \
scan_id_df[('day_baseline','recent_medical_history','tpa_datetime')].apply(convert_excel_time).dt.round('5min')

In [3]:
remove_tests = \
[('day_baseline','physical_examination'), 
 ('day_baseline', 'past_medical_history'), 
 ('day_baseline', 'CCMI'), 
 ('day_baseline', 'further_stroke_event_part'), 
 ('day_baseline', 'bloods'), 
 ('day_baseline', 'laboratory_tests'), 
 ('day_baseline', 'radiological_scans'), 
 ('day_baseline', 'concomitant_medications'), 
 ('day_baseline', 'pre-stroke_mRS'),
 ('day_7', 'bloods') , 
 ('day_7', 'history_of_depression'), 
 ('day_7', 'physical_risk_factors'), 
 ('day_7', 'diet_q'),
 ('day_7', 'laboratory_tests'), 
 ('day_7', 'concomitant_medications'), 
 ('day_7', 'concomitant_medications_plus'), 
 ('day_90', 'bloods'), 
 ('day_90', 'physical_examination'), 
 ('day_90', 'physical_risk_factors'), 
 ('day_90', 'diet_q'), 
 ('day_90', 'laboratory_tests'), 
 ('day_90', 'further_stroke_event_part'), 
 ('day_90', 'history_of_depression'), 
 ('day_90', 'concomitant_medications'), 
 ('day_365', 'bloods'), 
 ('day_365', 'physical_examination'), 
 ('day_365', 'physical_risk_factors'), 
 ('day_365', 'diet_q'), 
 ('day_365', 'laboratory_tests'), 
 ('day_365', 'further_stroke_event_part'), 
 ('day_365', 'history_of_depression'), 
 ('day_365', 'concomitant_medications'), 
 ('day_1', 'vital_signs'), 
 ('day_1', 'bloods'), 
 ('day_1', 'laboratory_tests'), 
 ('day_1', 'serious_adverse_event')]

scan_id_df.drop(remove_tests, axis = 'columns',inplace = True)
scan_id_df.columns = scan_id_df.columns.remove_unused_levels()

/Users/alistair/anaconda3/lib/python3.6/site-packages/ipykernel/kernelbase.py:399: PerformanceWarning: indexing past lexsort depth may impact performance.
  user_expressions, allow_stdin)


In [4]:
demographics = pd.read_pickle('base_df_data.p')


stroke_onset = demographics.loc[scans.index,['identity_stroke_onset']]
multi_NIHSS = scan_id_df[('day_baseline','NIHSS_multiple')]
multi_with_stroke_onset = stroke_onset.join(multi_NIHSS)

for c in ['nihss_three_hr_datetime', 'nihss_seven_hr_datetime', 
          'nihss_eighteen_hr_datetime', 'nihss_other_hr_datetime']:
    hrs = multi_with_stroke_onset[c] - multi_with_stroke_onset['identity_stroke_onset']
    multi_with_stroke_onset[c] = hrs.dt.round('5min')
    
scan_id_df[('day_baseline','NIHSS_multiple')] = multi_with_stroke_onset

/Users/alistair/anaconda3/lib/python3.6/site-packages/ipykernel/kernelbase.py:399: PerformanceWarning: indexing past lexsort depth may impact performance.
  user_expressions, allow_stdin)
/Users/alistair/anaconda3/lib/python3.6/site-packages/ipykernel/zmqshell.py:533: PerformanceWarning: indexing past lexsort depth may impact performance.
  return super(ZMQInteractiveShell, self).run_cell(*args, **kwargs)
/Users/alistair/anaconda3/lib/python3.6/site-packages/ipykernel/ipkernel.py:196: PerformanceWarning: indexing past lexsort depth may impact performance.
  res = shell.run_cell(code, store_history=store_history, silent=silent)


In [5]:
bit_1_cols = demographics.loc[:,'identity_gender':'identity_age_at_stroke_in_years'].columns
bit_2_cols = demographics.loc[:,'demographics_ancestry_1':].columns
useful_columns = bit_1_cols.append(bit_2_cols)
demographics = demographics.loc[scans.index,useful_columns]

In [6]:
demographics['identity_gender'] = demographics['identity_gender'].map({1:'female',0:'male'})

In [7]:
id_codes = pd.read_pickle('imaging_id_codes.p').loc[scans.index,'prefix']
infarct_type = scan_id_df[('day_1','infarct_type')]
scan_id_df[('day_90','lesion_location_confirmed')].join([id_codes,infarct_type])

/Users/alistair/anaconda3/lib/python3.6/site-packages/ipykernel/kernelbase.py:399: PerformanceWarning: indexing past lexsort depth may impact performance.
  user_expressions, allow_stdin)


,lesion location confirmed by imaging at 3 months (BC report),prefix,is_ich,date_time,symptomatic,loc_frontal,loc_temporal,loc_parietal,loc_occipital,loc_subcortical,loc_brainstem,loc_cerebellar,loc_subdural,loc_subarachnoid,treatment,treatment_spec
119,right,D_PP_P01-01-JW,0.0,NaT,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
123,right,D_PP_P02-01-AS,0.0,NaT,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
124,left,D_PP_P12-01-JW,0.0,NaT,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
126,right,D_PP_P01-10-NP,0.0,NaT,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
127,right,D_PP_P12-02-AS,0.0,NaT,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
129,right,D_PP_P01-07-GS,0.0,NaT,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
132,right,D_PP_P02-02-SH,0.0,NaT,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
133,left pons,D_PP_P01-11-GW,0.0,NaT,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
136,right,D_PP_P15-01-EM,0.0,NaT,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
137,right,D_PP_P15-02-DK,0.0,NaT,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN


In [27]:
scan_id_df[('day_90','TUGT')].head()

/Users/alistair/anaconda3/lib/python3.6/site-packages/ipykernel/kernelbase.py:399: PerformanceWarning: indexing past lexsort depth may impact performance.
  user_expressions, allow_stdin)


,time_taken
119,213.0
123,300.0
124,237.0
126,300.0
127,47.0


In [29]:
scan_id_df[('day_90','TUGT_average')].head()

/Users/alistair/anaconda3/lib/python3.6/site-packages/ipykernel/kernelbase.py:399: PerformanceWarning: indexing past lexsort depth may impact performance.
  user_expressions, allow_stdin)


,time1,time2,time3,TUGT_average
119,10.0,10.0,10.0,10.0
123,NaN,NaN,NaN,NaN
124,6.0,7.0,7.0,6.7
126,11.0,9.0,10.0,10.0
127,6.0,5.0,5.0,4.7


In [8]:
scan_id_df[('day_90','TUGT_average','correct_average')] = scan_id_df['day_90']['TUGT_average'][['time1', 'time2', 'time3']].mean(axis = 'columns').round(1)

In [9]:
scan_id_df['day_90']['TUGT_average'].head()

,time1,time2,time3,TUGT_average,correct_average
119,10.0,10.0,10.0,10.0,10.0
123,NaN,NaN,NaN,NaN,NaN
124,6.0,7.0,7.0,6.7,6.7
126,11.0,9.0,10.0,10.0,10.0
127,6.0,5.0,5.0,4.7,5.3


In [10]:
scan_id_df[('day_365','TUGT','correct_average')] = scan_id_df['day_365']['TUGT'][['time1', 'time2', 'time3']].mean(axis='columns').round(1)

In [11]:
scan_id_df['day_365']['TUGT'].head()

,time1,time2,time3,average,correct_average
119,NaN,NaN,NaN,NaN,NaN
123,30.0,30.0,30.0,30.0,30.0
124,8.0,8.0,7.0,7.3,7.7
126,9.0,9.0,10.0,8.7,9.3
127,6.0,6.0,6.0,5.7,6.0


In [75]:
scan_id_df['day_90']['TUGT'].head()

,time_taken
119,213.0
123,300.0
124,237.0
126,300.0
127,47.0


### Do it!

delete the individual columns and keep the calculated average

In [13]:
scan_id_df.filter(regex='TUGT').columns.tolist()

[('day_90', 'TUGT', 'time_taken'),
 ('day_90', 'TUGT_average', 'time1'),
 ('day_90', 'TUGT_average', 'time2'),
 ('day_90', 'TUGT_average', 'time3'),
 ('day_90', 'TUGT_average', 'TUGT_average'),
 ('day_365', 'TUGT', 'time1'),
 ('day_365', 'TUGT', 'time2'),
 ('day_365', 'TUGT', 'time3'),
 ('day_365', 'TUGT', 'average'),
 ('day_90', 'TUGT_average', 'correct_average'),
 ('day_365', 'TUGT', 'correct_average')]

In [14]:
bad = \
[('day_90', 'TUGT', 'time_taken'),
 ('day_90', 'TUGT_average', 'time1'),
 ('day_90', 'TUGT_average', 'time2'),
 ('day_90', 'TUGT_average', 'time3'),
 ('day_90', 'TUGT_average', 'TUGT_average'),
 ('day_365', 'TUGT', 'time1'),
 ('day_365', 'TUGT', 'time2'),
 ('day_365', 'TUGT', 'time3'),
 ('day_365', 'TUGT', 'average')]
scan_id_df.drop(bad,axis='columns',inplace=True)

In [16]:
# drop mri
mri_drop = scan_id_df.filter(regex='MRI').columns.tolist()
scan_id_df.drop(mri_drop,axis='columns',inplace=True)
scan_id_df.columns = scan_id_df.columns.remove_unused_levels()

In [26]:
response_drop = \
[('day_7', 'RAPA', 'resp'),
 ('day_7', 'RAPA', 'resp_other'),
 ('day_7', 'RAPA', 'resp_date'),
 ('day_90', 'mRS', 'resp'),
 ('day_90', 'mRS', 'resp_other'),
 ('day_90', 'BI', 'resp.1'),
 ('day_90', 'BI', 'resp_other.1'),
 ('day_90', 'SIS', 'resp.2'),
 ('day_90', 'SIS', 'resp_other.2'),
 ('day_90', 'RAPA', 'resp'),
 ('day_90', 'RAPA', 'resp_other'),
 ('day_90', 'RAPA', 'resp_date'),
 ('day_90', 'WSAS', 'resp.5'),
 ('day_90', 'WSAS', 'resp_other.5'),
 ('day_90', 'WSAS', 'resp_date.2'),
 ('day_365', 'mRS', 'resp'),
 ('day_365', 'mRS', 'resp_other'),
 ('day_365', 'BI', 'resp.1'),
 ('day_365', 'BI', 'resp_other.1'),
 ('day_365', 'SIS', 'resp.2'),
 ('day_365', 'SIS', 'resp_other.2'),
 ('day_365', 'RAPA', 'resp'),
 ('day_365', 'RAPA', 'resp_other'),
 ('day_365', 'RAPA', 'resp_date'),
 ('day_365', 'WSAS', 'resp.5'),
 ('day_365', 'WSAS', 'resp_other.5'),
 ('day_365', 'WSAS', 'resp_date.2')]

scan_id_df.drop(response_drop,axis='columns',inplace=True)
scan_id_df.columns = scan_id_df.columns.remove_unused_levels()

### Are these needed?

In [ ]:
('day_7', 'infarct_type')
('day_90', 'TDT', 'lesion_location_confirmed'),
('day_90', 'TDT', 'Leision_location'),
('day_90','lesion_location_confirmed','lesion location confirmed by imaging at 3 months (BC report)'),

In [32]:
scan_id_df[('day_7', 'infarct_type')].dropna(how='all')

/Users/alistair/anaconda3/lib/python3.6/site-packages/ipykernel/kernelbase.py:399: PerformanceWarning: indexing past lexsort depth may impact performance.
  user_expressions, allow_stdin)


,is_ich,date_time.1,symptomatic,loc_frontal,loc_temporal,loc_parietal,loc_occipital,loc_subcortical,loc_brainstem,loc_cerebellar,loc_subdural,loc_subarachnoid,treatment,treatment_spec
119,0.0,NaT,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
123,0.0,NaT,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
124,0.0,NaT,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
126,0.0,NaT,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
127,0.0,NaT,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
166,0.0,NaT,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
177,0.0,NaT,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
178,0.0,NaT,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN


In [33]:
scan_id_df[('day_90', 'TDT', 'Leision_location')]

119          0
123          0
124          1
126          0
127          0
129          0
132          0
133          1
136          0
137          0
138          1
139          0
140          1
141          0
142          2
143          2
144          0
146          0
147          0
148          1
151          0
152          1
154          1
155    1 and 2
156          1
157          1
159          0
161          2
162          0
163          1
        ...   
178          0
180          0
182          1
183          0
186          2
188          0
189          0
190          1
191          1
192          1
194          0
196          0
197          1
198          1
199          1
200          1
201          0
202          0
204          0
205          0
206          2
207          2
208          0
209          0
210          1
211          1
212          1
213          1
215          0
216          1
Name: (day_90, TDT, Leision_location), Length: 68, dtype: object

In [34]:
scan_id_df[('day_90','lesion_location_confirmed','lesion location confirmed by imaging at 3 months (BC report)')]

119                                                right
123                                                right
124                                                 left
126                                                right
127                                                right
129                                                right
132                                                right
133                                            left pons
136                                                right
137                                                right
138                                                 left
139                                                right
140                                                 left
141                                                right
142                                            no lesion
143                             Brainstem infarct (POCI)
144                                                right
146                            

In [35]:
infarct_type

,is_ich,date_time,symptomatic,loc_frontal,loc_temporal,loc_parietal,loc_occipital,loc_subcortical,loc_brainstem,loc_cerebellar,loc_subdural,loc_subarachnoid,treatment,treatment_spec
119,0.0,NaT,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
123,0.0,NaT,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
124,0.0,NaT,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
126,0.0,NaT,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
127,0.0,NaT,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
129,0.0,NaT,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
132,0.0,NaT,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
133,0.0,NaT,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
136,0.0,NaT,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
137,0.0,NaT,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN


In [51]:
scan_id_df.shape

(68, 1663)

In [59]:
scan_id_df[('day_baseline','NIHSS_multiple')].columns.tolist()

/Users/alistair/anaconda3/lib/python3.6/site-packages/ipykernel/kernelbase.py:399: PerformanceWarning: indexing past lexsort depth may impact performance.
  user_expressions, allow_stdin)


['nihss_three_hr',
 'nihss_three_hr_score',
 'nihss_three_hr_datetime',
 'nihss_seven_hr',
 'nihss_seven_hr_score',
 'nihss_seven_hr_datetime',
 'nihss_eighteen_hr',
 'nihss_eighteen_hr_score',
 'nihss_eighteen_hr_datetime',
 'nihss_other_hr',
 'nihss_other_hr_score',
 'nihss_other_hr_datetime']

In [61]:
drop_nihss_multiple = [('day_baseline','NIHSS_multiple',c) for c in ['nihss_three_hr',
                                                                     'nihss_seven_hr',
                                                                     'nihss_eighteen_hr',
                                                                     'nihss_other_hr']]

In [62]:
scan_id_df.drop(drop_nihss_multiple,axis = 'columns',inplace= True)

In [63]:
scan_id_df.columns = scan_id_df.columns.remove_unused_levels()

In [70]:
count_dict = scan_id_df.count().to_dict()

In [71]:
scan_id_df.shape

(68, 1659)

In [78]:
for k in count_dict.keys():
    print(k[0],'\n',k[1],'\n',k[2],'\n','\t\t\t\t',round((count_dict[k]/68)*100))

day_baseline 
 NIHSS_multiple 
 nihss_three_hr_score 
 				 24.0
day_baseline 
 NIHSS_multiple 
 nihss_three_hr_datetime 
 				 24.0
day_baseline 
 NIHSS_multiple 
 nihss_seven_hr_score 
 				 15.0
day_baseline 
 NIHSS_multiple 
 nihss_seven_hr_datetime 
 				 15.0
day_baseline 
 NIHSS_multiple 
 nihss_eighteen_hr_score 
 				 26.0
day_baseline 
 NIHSS_multiple 
 nihss_eighteen_hr_datetime 
 				 26.0
day_baseline 
 NIHSS_multiple 
 nihss_other_hr_score 
 				 31.0
day_baseline 
 NIHSS_multiple 
 nihss_other_hr_datetime 
 				 31.0
day_baseline 
 recent_medical_history 
 tpa 
 				 88.0
day_baseline 
 recent_medical_history 
 tpa_datetime 
 				 21.0
day_baseline 
 recent_medical_history 
 tpa_volume 
 				 21.0
day_baseline 
 NIHSS 
 loc 
 				 99.0
day_baseline 
 NIHSS 
 loc_quest 
 				 99.0
day_baseline 
 NIHSS 
 loc_command 
 				 99.0
day_baseline 
 NIHSS 
 best_gaze 
 				 99.0
day_baseline 
 NIHSS 
 vis_field_test 
 				 99.0
day_baseline 
 NIHSS 
 facial_palsy 
 				 99.0
day_

day_90 
 MADRS 
 pess_thought 
 				 100.0
day_90 
 MADRS 
 suicide_thought 
 				 100.0
day_90 
 MADRS 
 score 
 				 100.0
day_90 
 MADRS 
 valid 
 				 100.0
day_90 
 MADRS 
 reason 
 				 0.0
day_90 
 MMSE 
 age_assessment 
 				 100.0
day_90 
 MMSE 
 pencil 
 				 100.0
day_90 
 MMSE 
 watch 
 				 100.0
day_90 
 MMSE 
 school_years 
 				 99.0
day_90 
 MMSE 
 time_year 
 				 99.0
day_90 
 MMSE 
 time_season 
 				 99.0
day_90 
 MMSE 
 time_month 
 				 99.0
day_90 
 MMSE 
 time_weekday 
 				 99.0
day_90 
 MMSE 
 time_date 
 				 99.0
day_90 
 MMSE 
 place_state 
 				 99.0
day_90 
 MMSE 
 place_country 
 				 99.0
day_90 
 MMSE 
 place_city 
 				 99.0
day_90 
 MMSE 
 place_building 
 				 99.0
day_90 
 MMSE 
 place_floor 
 				 99.0
day_90 
 MMSE 
 register_apple 
 				 99.0
day_90 
 MMSE 
 register_penny 
 				 99.0
day_90 
 MMSE 
 register_table 
 				 99.0
day_90 
 MMSE 
 serial7_93 
 				 99.0
day_90 
 MMSE 
 serial7_86 
 				 99.0
day_90 
 MMSE 
 serial7_79 
 				 99.0
da

 grasp_ball_left 
 				 0.0
day_90 
 ARAT 
 grasp_ball_right 
 				 1.0
day_90 
 ARAT 
 grasp_stone_left 
 				 0.0
day_90 
 ARAT 
 grasp_stone_right 
 				 1.0
day_90 
 ARAT 
 grasp_subtotal_left 
 				 100.0
day_90 
 ARAT 
 grasp_subtotal_right 
 				 100.0
day_90 
 ARAT 
 grip_pour_water_left 
 				 100.0
day_90 
 ARAT 
 grip_pour_water_right 
 				 100.0
day_90 
 ARAT 
 grip_tube2_left 
 				 0.0
day_90 
 ARAT 
 grip_tube2_right 
 				 1.0
day_90 
 ARAT 
 grip_tube1_left 
 				 0.0
day_90 
 ARAT 
 grip_tube1_right 
 				 1.0
day_90 
 ARAT 
 grip_washer_left 
 				 0.0
day_90 
 ARAT 
 grip_washer_right 
 				 1.0
day_90 
 ARAT 
 grip_subtotal_left 
 				 100.0
day_90 
 ARAT 
 grip_subtotal_right 
 				 100.0
day_90 
 ARAT 
 pinch_small_ring_left 
 				 100.0
day_90 
 ARAT 
 pinch_small_ring_right 
 				 100.0
day_90 
 ARAT 
 pinch_small_middle_left 
 				 4.0
day_90 
 ARAT 
 pinch_small_middle_right 
 				 9.0
day_90 
 ARAT 
 pinch_small_index_left 
 				 4.0
day_90 
 ARAT 
 pinch_s

 leisure_1_86_pal 
 				 100.0
day_90 
 ACS 
 leisure_1_86_raln 
 				 100.0
day_90 
 ACS 
 leisure_1_86_ral 
 				 100.0
day_90 
 ACS 
 continue_HDL_1_82 
 				 100.0
day_90 
 ACS 
 givenup_HDL_1_82 
 				 100.0
day_90 
 ACS 
 beginning_HDL_1_82 
 				 100.0
day_90 
 ACS 
 new_HDL_1_82 
 				 100.0
day_90 
 ACS 
 HDL_1_82_caln 
 				 100.0
day_90 
 ACS 
 HDL_1_82_cal 
 				 100.0
day_90 
 ACS 
 HDL_1_82_pal 
 				 100.0
day_90 
 ACS 
 HDL_1_82_raln 
 				 100.0
day_90 
 ACS 
 HDL_1_82_ral3mth 
 				 100.0
day_90 
 ACS 
 continue_HDL_1_86 
 				 100.0
day_90 
 ACS 
 givenup_HDL_1_86 
 				 100.0
day_90 
 ACS 
 beginning_HDL_1_86 
 				 100.0
day_90 
 ACS 
 new_HDL_1_86 
 				 100.0
day_90 
 ACS 
 HDL_1_86_caln 
 				 100.0
day_90 
 ACS 
 HDL_1_86_cal 
 				 100.0
day_90 
 ACS 
 HDL_1_86_pal 
 				 100.0
day_90 
 ACS 
 HDL_1_86_raln 
 				 100.0
day_90 
 ACS 
 HDL_1_86_ral 
 				 100.0
day_90 
 ACS 
 continue_LDL_1_82 
 				 100.0
day_90 
 ACS 
 givenup_ LDL_1_82 
 				 100.0
day_90 
 A

day_365 
 MMSE 
 time_month 
 				 94.0
day_365 
 MMSE 
 time_weekday 
 				 94.0
day_365 
 MMSE 
 time_date 
 				 94.0
day_365 
 MMSE 
 place_state 
 				 94.0
day_365 
 MMSE 
 place_country 
 				 94.0
day_365 
 MMSE 
 place_city 
 				 94.0
day_365 
 MMSE 
 place_building 
 				 94.0
day_365 
 MMSE 
 place_floor 
 				 94.0
day_365 
 MMSE 
 register_apple 
 				 94.0
day_365 
 MMSE 
 register_penny 
 				 94.0
day_365 
 MMSE 
 register_table 
 				 94.0
day_365 
 MMSE 
 serial7_93 
 				 93.0
day_365 
 MMSE 
 serial7_86 
 				 93.0
day_365 
 MMSE 
 serial7_79 
 				 93.0
day_365 
 MMSE 
 serial7_72 
 				 93.0
day_365 
 MMSE 
 serial7_65 
 				 93.0
day_365 
 MMSE 
 world_d 
 				 4.0
day_365 
 MMSE 
 world_l 
 				 1.0
day_365 
 MMSE 
 world_r 
 				 1.0
day_365 
 MMSE 
 world_o 
 				 1.0
day_365 
 MMSE 
 world_w 
 				 1.0
day_365 
 MMSE 
 recall_apple 
 				 94.0
day_365 
 MMSE 
 recall_penny 
 				 94.0
day_365 
 MMSE 
 recall_table 
 				 94.0
day_365 
 MMSE 
 naming_pencil 


 ARAT 
 grip_pour_water_right 
 				 94.0
day_365 
 ARAT 
 grip_tube2_left 
 				 4.0
day_365 
 ARAT 
 grip_tube2_right 
 				 0.0
day_365 
 ARAT 
 grip_tube1_left 
 				 3.0
day_365 
 ARAT 
 grip_tube1_right 
 				 0.0
day_365 
 ARAT 
 grip_washer_left 
 				 3.0
day_365 
 ARAT 
 grip_washer_right 
 				 0.0
day_365 
 ARAT 
 grip_subtotal_left 
 				 96.0
day_365 
 ARAT 
 grip_subtotal_right 
 				 96.0
day_365 
 ARAT 
 pinch_small_ring_left 
 				 96.0
day_365 
 ARAT 
 pinch_small_ring_right 
 				 96.0
day_365 
 ARAT 
 pinch_small_middle_left 
 				 4.0
day_365 
 ARAT 
 pinch_small_middle_right 
 				 4.0
day_365 
 ARAT 
 pinch_small_index_left 
 				 4.0
day_365 
 ARAT 
 pinch_small_index_right 
 				 4.0
day_365 
 ARAT 
 pinch_large_ring_left 
 				 4.0
day_365 
 ARAT 
 pinch_large_ring_right 
 				 4.0
day_365 
 ARAT 
 pinch_large_middle_left 
 				 4.0
day_365 
 ARAT 
 pinch_large_middle_right 
 				 4.0
day_365 
 ARAT 
 pinch_large_index_left 
 				 4.0
day_365 
 ARAT 
 pinch_l

 ACS 
 household_1_82_caln 
 				 96.0
day_365 
 ACS 
 house_1_82_cal 
 				 96.0
day_365 
 ACS 
 household_1_82_pal 
 				 96.0
day_365 
 ACS 
 household_1_82_raln 
 				 96.0
day_365 
 ACS 
 household_1_82_ral 
 				 96.0
day_365 
 ACS 
 1_82_pal 
 				 96.0
day_365 
 ACS 
 1_82_caln 
 				 96.0
day_365 
 ACS 
 1_82_cal 
 				 96.0
day_365 
 ACS 
 1_82_raln 
 				 96.0
day_365 
 ACS 
 1_82_ral 
 				 96.0
day_365 
 ACS 
 1_82_lan 
 				 96.0
day_365 
 ACS 
 1_82_la 
 				 96.0
day_365 
 ACS 
 83_86_pal 
 				 87.0
day_365 
 ACS 
 83_86_caln 
 				 87.0
day_365 
 ACS 
 83_86_cal 
 				 87.0
day_365 
 ACS 
 83_86_raln 
 				 87.0
day_365 
 ACS 
 83_86_ral 
 				 87.0
day_365 
 ACS 
 83_86_lan 
 				 87.0
day_365 
 ACS 
 83_86_la 
 				 87.0
day_365 
 ACS 
 1_86_pal 
 				 96.0
day_365 
 ACS 
 continue_leisure_1_86 
 				 96.0
day_365 
 ACS 
 givenup_leisure_1_86 
 				 96.0
day_365 
 ACS 
 beginning_leisure_1_86 
 				 96.0
day_365 
 ACS 
 new_leisure_1_86 
 				 96.0
day_365 
 ACS 
 l

### No data in RAVENS

In [77]:
df[('day_365', 'RAVENs')]

/Users/alistair/anaconda3/lib/python3.6/site-packages/ipykernel/kernelbase.py:399: PerformanceWarning: indexing past lexsort depth may impact performance.
  user_expressions, allow_stdin)


,a1_response,a1_correct,a2_response,a2_correct,a3_response,a3_correct,a4_response,a4_correct,a5_response,a5_correct,...,b8_correct,b9_response,b9_correct,b10_response,b10_correct,b11_response,b11_correct,b12_response,b12_correct,score
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
